In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import matplotlib.pyplot as plt
from torchvision import models, transforms, datasets
from PIL import Image


In [2]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
print(DEVICE)

cuda


In [4]:

class ImageFolderWithPaths(datasets.ImageFolder):

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path


#print('00:\n')
#Image.open('train/train/00/00b32bd5ba9cdd7c2f11e3975b3e54fa.jpg')

In [5]:
#preprocess images
transforms_ = transforms.Compose([
    transforms.Resize(size=[224, 224], interpolation=2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

dataset = ImageFolderWithPaths('train/train/', transforms_) 

#define dataloader
BATCH = 1
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH)

In [6]:
import torchvision.models as models

pretrained_net = models.vgg13_bn(pretrained=True)


In [7]:



def pooling_output(x):
    #write your code here
  for layer_name, layer in pretrained_net._modules.items():
    #layer_name = layer_name.cuda()
    layer = layer.cuda()
    x = layer(x)
    if layer_name == 'avgpool':
        break
    #print(x.shape)
  return x.view(BATCH,-1)


In [ ]:


vec_img_list = []



with torch.no_grad():
    pretrained_net.eval()
    for item, (inputs, labels, paths) in enumerate(dataloader):
        print("Batch {}/{}".format(item,len(dataloader)), end='\r')
        inputs = inputs.to(DEVICE)
        
        img_vec = pooling_output(inputs)
        vec_img_list += [(img_vec.cpu(), paths[0])]
        torch.cuda.empty_cache()


In [ ]:

#query image
query_image = 'test/test/0a7cebae9c84961ea176d6083101f03e.jpg'
img = Image.open(query_image)

#transform the query img to vector and reshape it
input_tensor = transforms_(img)

input_tensor = input_tensor.view(1, *input_tensor.shape)



#obtain query_vector 
with torch.no_grad():
  input_tensor  = input_tensor.cuda()
  print(input_tensor.shape)
  query_vector = pooling_output(input_tensor)
  query_vector = query_vector.cpu()

In [ ]:
#reshape the query
query_vector = query_vector.reshape(1,25088) 

In [ ]:


import time


cos = nn.CosineSimilarity(dim=1, eps=1e-6)

#find top N images in the data that match query image.
N = 10
cosine_scores_list = []
cosine_scores = {}
start = time.time()

for i in range(len(vec_img_list)):
  temp_cos_score = cos(query_vector, vec_img_list[i][0])
  temp_cos_score = temp_cos_score.item()
  
  cosine_scores[temp_cos_score] = vec_img_list[i][1]
  #store in a list to sort 
  cosine_scores_list.append(temp_cos_score)


cosine_scores_list.sort(reverse= True)
print(time.time() - start)

 

In [ ]:
#pls run all the cells below 
print("Top N most similar images are ...")
x0 = cosine_scores[cosine_scores_list[0]]
Image.open(x0)



In [ ]:
x1 = cosine_scores[cosine_scores_list[1]]
Image.open(x1)

In [ ]:
x2 = cosine_scores[cosine_scores_list[2]]
Image.open(x2)

In [ ]:
x3 = cosine_scores[cosine_scores_list[3]]
Image.open(x3)

In [ ]:
x4 = cosine_scores[cosine_scores_list[4]]
Image.open(x4)

In [ ]:
x5 = cosine_scores[cosine_scores_list[5]]
Image.open(x5)

In [ ]:
x6 = cosine_scores[cosine_scores_list[6]]
Image.open(x6)

In [ ]:
x7 = cosine_scores[cosine_scores_list[7]]
Image.open(x7)

In [ ]:
x8 = cosine_scores[cosine_scores_list[8]]
Image.open(x8)

In [ ]:
x9 = cosine_scores[cosine_scores_list[9]]
Image.open(x9)